# Corruption Perception Index - Data prep-rocessing

By Jaime Lopez  
April 20, 2019

This is project to analyze results of Corruption Perception Index for different years. In this notebook data is obtained and pre-processed. At the end a unique dataset is built to be used in posterior analysis.

In [1]:
# Library setup
import pandas as pd
import requests
import zipfile

## Getting the data from Transparency Internacional

In this section data is downloaded from Transparency International's website. Data is in Excel format. For some years, data is packed in zip files.

In [2]:
# URL data sources
url = {
    '2012': 'http://files.transparency.org/content/download/533/2213/file/2012_CPI_DataPackage.zip',
    '2013': 'http://files.transparency.org/content/download/702/3015/file/CPI2013_DataBundle.zip',
    '2014': 'http://files.transparency.org/content/download/1857/12438/file/CPI2014_DataBundle.zip',
    '2015': 'http://files.transparency.org/content/download/1950/12812/file/2015_CPI_DataMethodologyZIP.zip',
    '2016': 'http://files.transparency.org/content/download/2155/13635/file/CPI2016_FullDataSetWithRegionalTables.xlsx',
    '2017': 'https://files.transparency.org/content/download/2172/13704/file/CPI2017_Full_DataSet_.xlsx',
    '2018': 'https://www.transparency.org/files/content/pages/2018_CPI_FullResults.zip'
}

In [3]:
# A temporal directory is created. Downloaded data will be placed here.
! rm -r -f tmp
! mkdir tmp

In [4]:
# Using a loop to get each file
files = []
for year in url.keys():
    # Filename is extracted from the last chunck in each url
    pos = url[year].rfind('/')
    filename = 'tmp/' + url[year][pos + 1:]
    try:
        # Get Request is made without SSL certificate verification
        data = requests.get(url[year], verify=False).content
        # Data is saved in local disk
        with open(filename, 'wb') as fd:
            fd.write(data)
        print(filename)
        files.append(filename)
    except requests.exceptions.RequestException as e:
        # If a any error ocurrs, display a message
        print(e)
        print('Failed downloading %s file' % filename)

tmp/2012_CPI_DataPackage.zip
tmp/CPI2013_DataBundle.zip
tmp/CPI2014_DataBundle.zip
tmp/2015_CPI_DataMethodologyZIP.zip
tmp/CPI2016_FullDataSetWithRegionalTables.xlsx


/usr/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


tmp/CPI2017_Full_DataSet_.xlsx


/usr/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


tmp/2018_CPI_FullResults.zip


In [5]:
# Checking list of downloaded files
! ls -l tmp

total 8344
-rw-r--r-- 1 jailop jailop 1772245 Apr 21 19:33 2012_CPI_DataPackage.zip
-rw-r--r-- 1 jailop jailop 1929095 Apr 21 19:33 2015_CPI_DataMethodologyZIP.zip
-rw-r--r-- 1 jailop jailop  154358 Apr 21 19:33 2018_CPI_FullResults.zip
-rw-r--r-- 1 jailop jailop 1815694 Apr 21 19:33 CPI2013_DataBundle.zip
-rw-r--r-- 1 jailop jailop 2631368 Apr 21 19:33 CPI2014_DataBundle.zip
-rw-r--r-- 1 jailop jailop  110876 Apr 21 19:33 CPI2016_FullDataSetWithRegionalTables.xlsx
-rw-r--r-- 1 jailop jailop  115964 Apr 21 19:33 CPI2017_Full_DataSet_.xlsx


In [6]:
# ZIP files are unzipped
for filename in files:
    if filename[-4:] == '.zip':
        zip_ref = zipfile.ZipFile(filename, 'r')
        zip_ref.extractall('tmp')
        zip_ref.close()

In [7]:
# Checking final list of downloaded files
! find tmp | sort

tmp
tmp/2012_CPI_DataPackage
tmp/2012_CPI_DataPackage/CPI2012_Results.xls
tmp/2012_CPI_DataPackage/Methodology notes
tmp/2012_CPI_DataPackage/Methodology notes/2012_CPIShortMethodologyNote_AR.pdf
tmp/2012_CPI_DataPackage/Methodology notes/2012_CPIShortMethodologyNote_EN.pdf
tmp/2012_CPI_DataPackage/Methodology notes/2012_CPIShortMethodologyNote_ES.pdf
tmp/2012_CPI_DataPackage/Methodology notes/2012_CPIShortMethodologyNote_FR.pdf
tmp/2012_CPI_DataPackage/Methodology notes/2012_CPIShortMethodologyNote_RU.pdf
tmp/2012_CPI_DataPackage/Methodology notes/2012_CPIShortMethodologyNote_ZH.pdf
tmp/2012_CPI_DataPackage/Methodology notes/2012_CPITechnicalMethodologyNote_AR.pdf
tmp/2012_CPI_DataPackage/Methodology notes/2012_CPITechnicalMethodologyNote_EN.pdf
tmp/2012_CPI_DataPackage/Methodology notes/2012_CPITechnicalMethodologyNote_ES.pdf
tmp/2012_CPI_DataPackage/Methodology notes/2012_CPITechnicalMethodologyNote_FR.pdf
tmp/2012_CPI_DataPackage/Methodology notes/2012_CPIUpdatedMethodology_AR.pdf


## Data Pre-processing

In this section data is read from Excel files(they were previously examined). Attribute names are uniformed and key attributes are selected for each year.

In [8]:
# All dataset will be stored in this dictionary
cpi = {}

In [9]:
cpi['2012'] = pd.read_excel('tmp/2012_CPI_DataPackage/CPI2012_Results.xlsx', skiprows=1)
cpi['2013'] = pd.read_excel('tmp/CPI2013_DataBundle/CPI2013_GLOBAL_WithDataSourceScores.xls', skiprows=2)
cpi['2014'] = pd.read_excel('tmp/CPI2014_DataBundle/CPI 2014_FullDataSet.xlsx')
cpi['2015'] = pd.read_excel('tmp/2015_CPI_DataMethodologyZIP/Data/CPI_2015_FullDataSet.xlsx', skiprows=0)
cpi['2016'] = pd.read_excel('tmp/CPI2016_FullDataSetWithRegionalTables.xlsx')
cpi['2017'] = pd.read_excel('tmp/CPI2017_Full_DataSet_.xlsx', skiprows=2)
cpi['2018'] = pd.read_excel('tmp/2018_CPI_FullDataSet.xlsx', sheet_name='CPI2018', skiprows=2)

Data is loaded. Now, attributes are uniformed and selected for each year.

In [10]:
year = '2012'
cpi[year].rename(columns={
    'Unnamed: 1': 'Country', 
    'Unnamed: 2': 'Region', 
    'Unnamed: 3': 'Score', 
    'Unnamed: 5': 'Surveys',
    'Unnamed: 6': 'Error',
}, inplace=True)
cpi[year] = cpi[year][['Country', 'Region', 'Score', 'Surveys', 'Error', 'Lower', 'Upper']]
cpi[year]['year'] = year
cpi[year].head()

,Country,Region,Score,Surveys,Error,Lower,Upper,year
0,Denmark,EU,90,7,2.0,87,93,2012
1,Finland,EU,90,7,3.0,85,95,2012
2,New Zealand,AP,90,7,2.2,87,94,2012
3,Sweden,EU,88,7,1.9,85,91,2012
4,Singapore,AP,87,9,2.1,83,90,2012


In [11]:
year = '2013'
cpi[year].rename(columns={
    'Unnamed: 1': 'Country', 
    'Unnamed: 2': 'Region', 
    'Unnamed: 6': 'Score', 
    'Unnamed: 7': 'Surveys',
    'Unnamed: 8': 'Error',
}, inplace=True)
cpi[year] = cpi[year][['Country', 'Region', 'Score', 'Surveys', 'Error', 'Lower', 'Upper']]
cpi[year]['year'] = year
cpi[year].head()

,Country,Region,Score,Surveys,Error,Lower,Upper,year
0,Denmark,EU,91,7,2.2,87,95,2013
1,New Zealand,AP,91,7,2.3,87,95,2013
2,Finland,EU,89,7,1.7,86,92,2013
3,Sweden,EU,89,7,2.3,85,93,2013
4,Norway,EU,86,7,2.3,82,90,2013


In [12]:
year = '2014'
cpi[year].rename(columns={
    'Country/Territory': 'Country', 
    'CPI 2014': 'Score', 
    'Number of Surveys Used': 'Surveys',
    'Std Error': 'Error',
    '90% Lower CI': 'Lower',
    '90% Upper CI': 'Upper'
}, inplace=True)
cpi[year] = cpi[year][['Country', 'Region', 'Score', 'Surveys', 'Error', 'Lower', 'Upper']]
cpi[year]['year'] = year
cpi[year].head()

,Country,Region,Score,Surveys,Error,Lower,Upper,year
0,Denmark,EU,92,7,2.04,89,95,2014
1,New Zealand,AP,91,7,2.28,87,95,2014
2,Finland,EU,89,7,2.05,86,92,2014
3,Sweden,EU,87,7,3.41,81,93,2014
4,Norway,EU,86,7,2.38,82,90,2014


In [13]:
year = '2015'
cpi[year].rename(columns={
    'Country/Territory': 'Country', 
    'CPI 2015 Score': 'Score', 
    'Number of Surveys Used': 'Surveys',
    'Std Error': 'Error',
    '90% Lower CI': 'Lower',
    '90%Upper CI': 'Upper'
}, inplace=True)
cpi[year] = cpi[year][['Country', 'Region', 'Score', 'Surveys', 'Error', 'Lower', 'Upper']]
cpi[year]['year'] = year
cpi[year].head()

,Country,Region,Score,Surveys,Error,Lower,Upper,year
0,Denmark,WE/EU,91,7,2.16,87,95,2015
1,New Zealand,AP,91,7,2.32,87,95,2015
2,Finland,WE/EU,90,7,1.77,87,93,2015
3,Sweden,WE/EU,89,7,1.71,86,92,2015
4,Norway,WE/EU,88,7,2.24,84,92,2015


In [14]:
year = '2016'
cpi[year].rename(columns={
    'CPI2016': 'Score', 
    'Number of Sources': 'Surveys',
    'Std Error 2016': 'Error',
    'Lower CI': 'Lower',
    'Upper CI': 'Upper'
}, inplace=True)
cpi[year] = cpi[year][['Country', 'Region', 'Score', 'Surveys', 'Error', 'Lower', 'Upper']]
cpi[year]['year'] = year
cpi[year].head()

,Country,Region,Score,Surveys,Error,Lower,Upper,year
0,New Zealand,AP,90,7,2.56,86,94,2016
1,Denmark,WE/EU,90,7,2.46,86,94,2016
2,Finland,WE/EU,89,7,1.46,87,92,2016
3,Sweden,WE/EU,88,7,1.33,85,90,2016
4,Switzerland,WE/EU,86,6,1.57,83,89,2016


In [15]:
year = '2017'
cpi[year].rename(columns={
    'CPI Score 2017': 'Score', 
    'number of sources': 'Surveys',
    'standard error': 'Error',
    'Lower CI ': 'Lower',
    'Upper CI': 'Upper'
}, inplace=True)
cpi[year] = cpi[year][['Country', 'Region', 'Score', 'Surveys', 'Error', 'Lower', 'Upper']]
cpi[year]['year'] = year
cpi[year].head()

,Country,Region,Score,Surveys,Error,Lower,Upper,year
0,New Zealand,AP,89.0,8.0,2.40,85.0,93.0,2017
1,Denmark,WE/EU,88.0,8.0,2.75,83.0,93.0,2017
2,Finland,WE/EU,85.0,8.0,2.84,80.0,90.0,2017
3,Norway,WE/EU,85.0,8.0,1.83,82.0,88.0,2017
4,Switzerland,WE/EU,85.0,7.0,1.71,82.0,88.0,2017


In [16]:
year = '2018'
cpi[year].rename(columns={
    'CPI Score 2018': 'Score', 
    'Number of sources': 'Surveys',
    'Standard error': 'Error',
    'Lower CI ': 'Lower',
    'Upper CI': 'Upper'
}, inplace=True)
cpi[year] = cpi[year][['Country', 'Region', 'Score', 'Surveys', 'Error', 'Lower', 'Upper']]
cpi[year]['year'] = year
cpi[year].head()

,Country,Region,Score,Surveys,Error,Lower,Upper,year
0,Denmark,WE/EU,88,8,2.63,84,92,2018
1,New Zealand,AP,87,8,2.44,83,91,2018
2,Finland,WE/EU,85,8,2.74,81,89,2018
3,Singapore,AP,85,9,1.98,82,88,2018
4,Sweden,WE/EU,85,8,2.02,82,88,2018


## Creating a unique dataset

Finally, a unique dataset is built joining datasets for each year. The new dataset is stored in the `cpi.csv` file.

In [17]:
data = pd.concat(cpi)

In [18]:
data.to_csv('cpi.csv', index=False)

# Cleaning work area

The `tmp` directory is deleted.

In [20]:
! rm -rf tmp